# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
import os
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/jack/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/jack/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/decl-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/

### Running through an example

In [2]:
# EW vulnerable.
vuln_ns, vuln_ew = False, True
# you sit North and hold:
hand = 'QT5.T8.KQT87.864'
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 0, 3, False)

# the auction goes:
auction = ["PAD_START","PAD_START","PAD_START","PASS"]
bid = bot_bid.bid(auction)

auction = ["PAD_START","PAD_START","PAD_START","PASS", "PASS", "1N", "2C", "PASS"]
# what's your bid?
bid = bot_bid.bid(auction)


Loaded lib dds.dll


In [3]:
bid.samples

['QTx.T8.KQT8x.8xx AKxx.AKQx.J9x.xx xxx.J9x.Ax.AKQT9 J98.xxxx.xxx.Jxx 0.88559 \n P-P-1N-2C-P-2S-P-P-P (6, 6.0, 7.0)  \n P-P-1N-2C-P-P-P (9, 8.3, 8.5)  \n P-P-1N-2C-P-2H-P-P-P (6, 5.5, 7.2) ',
 'QTx.T8.KQT8x.8xx AKxx.Qxxx.Ax.AT9 Jxx.AK9.9x.KQJxx 98x.Jxxx.Jxxx.xx 0.86874 \n P-P-1N-2C-P-2S-P-P-P (6, 6.7, 7.8)  \n P-P-1N-2C-P-P-P (9, 8.3, 8.1)  \n P-P-1N-2C-P-2H-P-P-P (5, 5.5, 7.6) ',
 'QTx.T8.KQT8x.8xx KJx.Axx.Axx.KQTx A98xx.KQ9xx.x.J9 xx.Jxx.J9xx.Axxx 0.86392 \n P-P-1N-2C-P-2S-P-P-P (7, 7.9, 7.9)  \n P-P-1N-2C-P-P-P (4, 4.3, 5.6)  \n P-P-1N-2C-P-2H-P-P-P (6, 6.8, 7.5) ',
 'QTx.T8.KQT8x.8xx Axx.AKJx.AJ9x.xx K98.9xx.xx.AKQ9x Jxxx.Qxxx.xx.JTx 0.85282 \n P-P-1N-2C-P-2S-P-P-P (6, 5.5, 6.3)  \n P-P-1N-2C-P-P-X-P-2H-P-P-P (8, 8.4, 8.7)  \n P-P-1N-2C-P-2H-P-P-P (5, 4.7, 6.2) ',
 'QTx.T8.KQT8x.8xx xxx.AQx.AJ9x.KJx AKJxx.KJxxx.x.Qx 98.9xx.xxx.AT9xx 0.82567 \n P-P-1N-2C-P-2S-P-P-P (9, 8.8, 8.6)  \n P-P-1N-2C-P-P-P (4, 4.6, 6.5)  \n P-P-1N-2C-P-2H-P-P-P (8, 7.6, 7.7) ',
 'QTx.T8.KQT8x.8xx Kx.KJx.Axx

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [4]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(["PAD_START","PAD_START","PAD_START","PASS", "PASS", "1N", "2C", "PASS"],0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

Good quality: True
QTx.T8.KQT8x.8xx AKxx.AKQx.J9x.xx xxx.J9x.Ax.AKQT9 J98.xxxx.xxx.Jxx
QTx.T8.KQT8x.8xx AKxx.Qxxx.Ax.AT9 Jxx.AK9.9x.KQJxx 98x.Jxxx.Jxxx.xx
QTx.T8.KQT8x.8xx KJx.Axx.Axx.KQTx A98xx.KQ9xx.x.J9 xx.Jxx.J9xx.Axxx
QTx.T8.KQT8x.8xx Axx.AKJx.AJ9x.xx K98.9xx.xx.AKQ9x Jxxx.Qxxx.xx.JTx
QTx.T8.KQT8x.8xx xxx.AQx.AJ9x.KJx AKJxx.KJxxx.x.Qx 98.9xx.xxx.AT9xx
QTx.T8.KQT8x.8xx Kx.KJx.Axxx.KJ9x A8xxx.Qxxxx.9.AQ J9x.A9x.Jxx.Txxx
QTx.T8.KQT8x.8xx A8xx.AQxx.Ax.KTx Kx.K9x.xxx.AQJxx J9xx.Jxxx.J9x.9x
QTx.T8.KQT8x.8xx Axx.AKxx.Axxx.Qx K9x.Qxxx..AKT9xx J8xx.J9x.J9xx.Jx
QTx.T8.KQT8x.8xx Jxx.AKJx.Axx.ATx AK9.9xx.J.KQJxxx 8xxx.Qxxx.9xxx.9
QTx.T8.KQT8x.8xx A9xx.KQJx.Axx.Kx KJx.A9xx.x.ATxxx 8xx.xxx.J9xx.QJ9
QTx.T8.KQT8x.8xx 9xxx.AKQx.Axx.QT AJ8.Jxxx.x.AK9xx Kxx.9xx.J9xx.Jxx
QTx.T8.KQT8x.8xx AJ.Qxxx.AJ9x.K9x Kxxxx.AKxx.x.JTx 98x.J9x.xxx.AQxx
QTx.T8.KQT8x.8xx AJ8x.AJx.xx.AK9x Kx.KQxx.Ax.QJTxx 9xxx.9xxx.J9xx.x
QTx.T8.KQT8x.8xx AJ8.AKx.Jxxx.Axx K9xx.Q9xxx.A.QTx xxx.Jxx.9xx.KJ9x
QTx.T8.KQT8x.8xx AKxx.Q9xx.Ax